# 인스타그램 태그 크롤링

- 현재 정보는 태그가 없는 경우가 있다.
- 그 태그 정보를 모두 크롤링하여 보자.

In [1]:
import pandas as pd
import numpy as np
import re 
import os

In [20]:
# directory  = 'C:/Users/USER/Documents/Jupyter_Notebook/KUAIA_Hackerton_3/인플루언서정보_샘플1/'
 
# file_list = os.listdir(directory)

In [22]:
# for file in file_list[0:1]:
#     df = pd.read_json(directory + file + '/data.json')


In [24]:
# df.iloc[-1,-2]

# 무신사 크롤링

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
large = ['americancasual','casual','chic','dandy','formal','girlish','golf','homewear','retro','romantic','sports','street']

In [12]:
from fake_useragent import UserAgent
import tqdm 


page_num = 4

photo_url_lst = []
ua = UserAgent()

headers = {
    'User-Agent': ua.random,
}

dict_all = {}

for small in tqdm.tqdm(large):

    url = 'https://store.musinsa.com/app/codimap/lists?style_type='+small+'&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page='+str(page_num)
    res = requests.get(url, headers = headers)
    
    soup = BeautifulSoup(res.content, 'html.parser')
    
    lst_style = []
    for sip in soup.select('div.style-list-information'):
        numbers = re.sub(r'[^0-9]', '', sip.strong['onclick'].split()[0])
        style_number = numbers
        style_name = sip.span.text
        style_name_part = sip.strong.text

        lst_style.append([style_number,style_name,style_name_part])

    df_style = pd.DataFrame(lst_style)

    url_2 = 'https://store.musinsa.com/app/codimap/views'

    ua_2 = UserAgent()

    headers_2 = {
        'User-Agent': ua_2.random,
    }
    
    for idx,num in enumerate(df_style[0].values):
        

        res_2 = requests.get(url_2 + '/' + str(num), headers = headers_2)
        soup_2 = BeautifulSoup(res_2.content, 'html.parser')

        lst_codimap = []
        for sip2 in soup_2.select('div.styling_tag'):
            lst_shap = []
            for sip2_small in sip2.select('a.ui-tag-list__item'):
                lst_shap.append(sip2_small.text.replace('#',''))

            lst_codimap.append([sip2.p.text, lst_shap])   


        for sip2_1 in soup_2.select('div.swiper-slide.style_contents_size'):
            url_3 = 'https:' + sip2_1.img['src']
            photo_url = url_3.replace('220.jpg','500.jpg')
            photo_url_lst.append([photo_url, df_style[1][idx], df_style[2][idx], url_2 + '/' + str(num),
                                  lst_codimap[0][0], lst_codimap[0][1]])
            
 
            

            

            
            
        

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [03:28<00:00, 17.39s/it]


In [13]:
df_total_2 = pd.DataFrame(photo_url_lst,columns = ['image_url','style','style_part','domain_url','comment','tag'])

In [15]:
# df_total_2.to_excel('Fourth_Page.xlsx' ,encoding= 'utf-8-sig')

In [73]:
# pd.concat([df,df_total_2]).to_excel('Third_Page.xlsx' ,encoding= 'utf-8-sig')

# 사진 받아오기

In [9]:
import os
import time
import pandas as pd
import numpy as np